----
# Upload Single Cell data to Broad Single Cell Portal
L.Richards  
April 2020  

----

Need to upload clustering, metadata and normalized expression matrix files to Broad Single Cell portal.Uploading in-house scRNA-seq data of MB cell lines only. In the same configuration as Figure 2A of Branavans paper.

>https://singlecell.broadinstitute.org/single_cell/study/SCP840: this link should be added to data availability section of manuscript
>  sheilasinghlab@gmail.com & chitrave@gmail.com have login and edit access to account

**Currently the study is private, but upon publication this accession should be made public**

/mnt/work1/users/pughlab/projects/SinghLab_scMedullo/Brananvan_Wnt/Broad_scRNA_Portal

In [ ]:
library("data.table")
library("dplyr")
library("taRifx")
library("Matrix")
suppressMessages(library("Seurat", 
                         lib ="/mnt/work1/users/pughlab/projects/BTSCs_scRNAseq/Manuscript/lib/" ))



SeuratPath <- "~/pughlab/projects/SinghLab_scMedullo/Brananvan_Wnt/AddModuleScore/WntMedullo_newQC_scoring.RData"
sample <- "Singh_BTIC_Medulloblastoma"



################################################

print("")
print("-------")
print(sample)
start.time <- Sys.time()
print(start.time)
print("-------")

################################################
print("")
print("")
print("Loading Seurat Object")
print(Sys.time())
print(SeuratPath)

loadRData <- function(fileName){
#loads an RData file, and returns it
    load(fileName)
    get(ls()[ls() != "fileName"])
}
      
dat <- loadRData(SeuratPath)
print(dim(dat@data))

################################################
print("")
print("")
print("Extract Normalized Gene Expression File (txt.gz)")
print(Sys.time())

norm.exp <- data.frame(as.matrix(dat@data))
norm.exp <- tibble::rownames_to_column(norm.exp, "GENE")


print(norm.exp[1:5, 1:5])

file1 <- paste0(sample, "_NormalizedExpression.txt.gz")
print(file1)
      
write.table(norm.exp, 
          file=gzfile(file1),
          row.names = FALSE,
          quote = FALSE,
            sep = "\t"
          )      


################################################
print("")
print("")
print("Extract Meta Data (.txt)")
print(Sys.time())

meta <- data.frame(dat@meta.data)
meta <- remove.factors(meta)

include <- c("nGene",
             "nUMI",
             "percent.mito",
             "Num.Reads",
             "Sample",
             "Subgroup",
             "Stage",
             colnames(dat@meta.data)[grep("GSVA.scaled", colnames(dat@meta.data))]
            )
meta <- meta[,include]
colnames(meta) <- gsub("GSVA.scaled", "GSVA", colnames(meta))
meta$nGene <- as.numeric(meta$nGene)
meta$nUMI <- as.numeric(meta$nUMI)
meta$nUMI <- as.numeric(meta$Num.Reads)

classes <- as.vector(unlist(lapply(meta, class)))
TYPE <- c("TYPE", ifelse(classes == "numeric", "numeric", "group"))
meta <- data.frame(setDT(meta, keep.rownames = "NAME"))
meta <- rbind(TYPE, meta)
      
file2 <- paste0(sample, "_MetaData.txt")
print(file2)

write.table(meta, 
          file=file2,
          row.names = FALSE,
          quote = FALSE,
          sep = "\t",
          col.names = TRUE
          ) 


################################################
print("")
print("")
print("Extract tSNE Coordinates (.txt)")
print(Sys.time())

cluster.coords <- as.data.frame(dat@dr$tsne@cell.embeddings)[, c(1:2)]
colnames(cluster.coords) <- c("X", "Y")
cluster.coords <- remove.factors(cluster.coords)

classes <- as.vector(unlist(lapply(cluster.coords, class)))
TYPE <- c("TYPE", ifelse(classes == "numeric", "numeric"))
cluster.coords <- data.frame(setDT(cluster.coords, keep.rownames = "NAME"))
cluster.coords  <- rbind(TYPE, cluster.coords)

##subset out some meta data to include as labels
cluster.coords <- cbind(cluster.coords, meta[ ,c("Sample",
                                                 "Subgroup",
                                                 "Stage"
                                                )] )

file3 <- paste0(sample, "_tSNE_ClusterCoordinates.txt")
print(file3)
write.table(cluster.coords, 
          file=file3,
          row.names = FALSE,
          quote = FALSE,
           sep = "\t",
          col.names = TRUE
          ) 
